# Diagram Subgraph Testing and Debugging

This notebook helps test and debug the diagram generation subgraph to identify issues with validation logic and improve diagram quality.

## Setup and Imports


In [1]:
import sys
import os
import json
from typing import Dict, Any, List
from pprint import pprint
from pathlib import Path

# Fix import paths - handle both running from root and from tests directory
current_dir = Path.cwd()
if current_dir.name == 'tests':
    # Running from tests directory
    backend_dir = current_dir.parent
    project_root = backend_dir.parent
else:
    # Running from project root
    backend_dir = current_dir / 'backend'
    project_root = current_dir

sys.path.insert(0, str(backend_dir))
sys.path.insert(0, str(project_root))

print(f"Current directory: {current_dir}")
print(f"Backend directory: {backend_dir}")
print(f"Project root: {project_root}")

# Import our modules
try:
    from workflows.diagram_subgraph import (
        create_diagram_subgraph, 
        _parser_errors_python_only,
        _edge_warnings,
        _label_warnings,
        _node_id_warnings,
        _complexity_warnings,
        _apply_correction
    )
    from workflows.state import DiagramState
    from services.content_generation import ContentGenerationService
    from utils.logger import get_logger
    
    logger = get_logger(__name__)
    content_service = ContentGenerationService()
    
    print("✅ Imports successful")
    
except ImportError as e:
    print(f"❌ Import error: {e}")
    print("Available modules in backend:")
    backend_modules = [f for f in os.listdir(backend_dir) if os.path.isdir(os.path.join(backend_dir, f)) and not f.startswith('.')]
    print(backend_modules)


Current directory: /home/shikhar/.a/.b/.c/code-architecture-mapper/backend/tests
Backend directory: /home/shikhar/.a/.b/.c/code-architecture-mapper/backend
Project root: /home/shikhar/.a/.b/.c/code-architecture-mapper


/home/shikhar/.a/.b/.c/code-architecture-mapper/backend/.venv/lib/python3.11/site-packages/tree_sitter/__init__.py:36: FutureWarning: Language(path, name) is deprecated. Use Language(ptr, name) instead.
  warn("{} is deprecated. Use {} instead.".format(old, new), FutureWarning)


2025-09-21 20:32:13,741 - workflows.graph - INFO - ✅ Analysis graph structure created successfully
2025-09-21 20:32:13,763 - workflows.graph - INFO - 🚀 AnalysisWorkflow initialized
✅ Imports successful


/home/shikhar/.a/.b/.c/code-architecture-mapper/backend/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Sample Test Data

Let's create some sample data to test with:


In [2]:
# Sample dependency analysis data
sample_dependency_analysis = {
    "imports": {
        "main.py": ["utils.py", "config.py"],
        "utils.py": ["helpers.py"],
        "config.py": []
    },
    "modules": ["main", "utils", "config", "helpers"]
}

# Sample JSON graph
sample_json_graph = {
    "nodes": [
        {"id": "main", "label": "main.py", "type": "file"},
        {"id": "utils", "label": "utils.py", "type": "file"},
        {"id": "config", "label": "config.py", "type": "file"},
        {"id": "helpers", "label": "helpers.py", "type": "file"}
    ],
    "edges": [
        {"source": "main", "target": "utils", "type": "import"},
        {"source": "main", "target": "config", "type": "import"},
        {"source": "utils", "target": "helpers", "type": "import"}
    ]
}

# Sample file infos
sample_file_infos = [
    {"path": "main.py", "size": 1500, "lines": 50},
    {"path": "utils.py", "size": 800, "lines": 30},
    {"path": "config.py", "size": 200, "lines": 10},
    {"path": "helpers.py", "size": 600, "lines": 25}
]

# Sample architecture markdown
sample_architecture_md = """
# Architecture Overview

This is a simple Python application with the following structure:
- main.py: Entry point
- utils.py: Utility functions
- config.py: Configuration settings
- helpers.py: Helper functions
"""

print("✅ Sample data created")


✅ Sample data created


## Test Individual Validation Functions

Let's test each validation function with different diagram examples:


In [3]:
# Test diagrams with different issues - Updated for new validation approach
test_diagrams = {
    "good_diagram": """
graph LR
    A[main.py] --> B[utils.py]
    A --> C[config.py]
    B --> D[helpers.py]
""".strip(),
    
    "missing_type": """
    A[main.py] --> B[utils.py]
    A --> C[config.py]
    B --> D[helpers.py]
""".strip(),
    
    "unbalanced_subgraph": """
graph LR
    subgraph "Main Module"
        A[main.py] --> B[utils.py]
    end
    subgraph "Config Module"
        C[config.py]
    end
""".strip(),
    
    "quoted_labels": """
graph LR
    A[main.py] -- "Uses configuration" --> B[config.py]
    A --|"Helper functions"|--> C[utils.py]
    B -.-> D[helpers.py]
""".strip(),
    
    "special_chars_valid": """
graph LR
    A[main.py & utils] --> B[config < settings]
    A --> C["quoted & special"]
    B --> D[helpers.py]
""".strip(),
    
    "long_labels": """
graph LR
    A[This is a very long label that exceeds the fifty character limit and should trigger validation error] --> B[utils.py]
    A --> C[config.py]
""".strip(),
    
    "complex_diagram": """
graph LR
    A[main.py] --> B[utils.py]
    A --> C[config.py]
    B --> D[helpers.py]
    C --> E[database.py]
    D --> F[models.py]
    E --> G[queries.py]
    F --> H[schema.py]
    G --> I[connection.py]
    H --> J[migrations.py]
    I --> K[pool.py]
    J --> L[seeds.py]
    K --> M[transactions.py]
    L --> N[validators.py]
    M --> O[serializers.py]
    N --> P[controllers.py]
    O --> Q[routes.py]
    P --> R[middleware.py]
    Q --> S[auth.py]
    R --> T[permissions.py]
    S --> U[tokens.py]
    T --> V[sessions.py]
    U --> W[cache.py]
    V --> X[logging.py]
    W --> Y[monitoring.py]
    X --> Z[metrics.py]
""".strip()
}

print("✅ Test diagrams created")
print(f"Number of test diagrams: {len(test_diagrams)}")

# Test each validation function with new approach
def test_validation_functions():
    print("\n🔍 Testing New Validation Functions\n")
    
    for name, diagram in test_diagrams.items():
        print(f"\n📊 Testing: {name}")
        print(f"Diagram preview: {diagram[:100]}...")
        
        # Test new validation approach
        parse_errors = _parser_errors_python_only(diagram)
        edge_warnings = _edge_warnings(diagram)
        label_warnings = _label_warnings(diagram)
        node_warnings = _node_id_warnings(diagram)
        complexity_warnings = _complexity_warnings(diagram)
        
        all_warnings = edge_warnings + label_warnings + node_warnings + complexity_warnings
        
        print(f"  Parse errors (blocking): {len(parse_errors)}")
        print(f"  Edge warnings: {len(edge_warnings)}")
        print(f"  Label warnings: {len(label_warnings)}")
        print(f"  Node warnings: {len(node_warnings)}")
        print(f"  Complexity warnings: {len(complexity_warnings)}")
        print(f"  Total warnings: {len(all_warnings)}")
        
        if parse_errors:
            print("  ❌ Blocking errors found:")
            for error in parse_errors:
                print(f"    - {error}")
        else:
            print("  ✅ No blocking errors - diagram will render!")
        
        if all_warnings:
            print("  ⚠️ Warnings found (non-blocking):")
            for warning in all_warnings[:3]:  # Show first 3 warnings
                print(f"    - {warning}")
        else:
            print("  ✅ No warnings found")
        
        print("-" * 50)

test_validation_functions()


✅ Test diagrams created
Number of test diagrams: 7

🔍 Testing New Validation Functions


📊 Testing: good_diagram
Diagram preview: graph LR
    A[main.py] --> B[utils.py]
    A --> C[config.py]
    B --> D[helpers.py]...
  Parse errors (blocking): 0
  Edge warnings: 3
  Label warnings: 0
  Node warnings: 0
  Complexity warnings: 0
  Total warnings: 3
  ✅ No blocking errors - diagram will render!
  ⚠️ Warnings found (non-blocking):
    - Line 2: edge looks unusual for flowchart syntax
    - Line 3: edge looks unusual for flowchart syntax
    - Line 4: edge looks unusual for flowchart syntax
--------------------------------------------------

📊 Testing: missing_type
Diagram preview: A[main.py] --> B[utils.py]
    A --> C[config.py]
    B --> D[helpers.py]...
  Parse errors (blocking): 1
  Edge warnings: 3
  Label warnings: 0
  Node warnings: 0
  Complexity warnings: 0
  Total warnings: 3
  ❌ Blocking errors found:
    - Missing diagram type declaration. Add 'flowchart LR' or 'graph TD' at

## Test Full Diagram Subgraph

Now let's test the complete diagram subgraph workflow:


In [4]:
def test_diagram_subgraph(mode="balanced"):
    print(f"🎨 Testing Full Diagram Subgraph - Mode: {mode}\n")
    
    # Create diagram subgraph
    diagram_graph = create_diagram_subgraph()
    
    # Create initial state
    initial_state: DiagramState = {
        "dependency_analysis": sample_dependency_analysis,
        "json_graph": sample_json_graph,
        "file_infos": sample_file_infos,
        "architecture_markdown": sample_architecture_md,
        "current_attempt": 0,
        "max_attempts": 3,
        "diagram_mode": mode,
        "raw_diagram": None,
        "validated_diagram": None,
        "validation_errors": [],
        "validation_warnings": [],  # New field
        "is_valid": False,
        "diagrams": {}
    }
    
    print(f"Initial state created for mode: {mode}")
    print(f"Max attempts: {initial_state['max_attempts']}")
    
    try:
        # Execute the subgraph
        final_state = diagram_graph.invoke(initial_state)
        
        print(f"\n📊 Subgraph execution completed!")
        print(f"Final attempt: {final_state['current_attempt']}")
        print(f"Is valid: {final_state['is_valid']}")
        print(f"Validation errors: {len(final_state['validation_errors'])}")
        print(f"Validation warnings: {len(final_state['validation_warnings'])}")
        
        if final_state['validation_errors']:
            print("\n❌ Blocking validation errors:")
            for i, error in enumerate(final_state['validation_errors'], 1):
                print(f"  {i}. {error}")
        
        if final_state['validation_warnings']:
            print("\n⚠️ Non-blocking validation warnings:")
            for i, warning in enumerate(final_state['validation_warnings'], 1):
                print(f"  {i}. {warning}")
        
        # Show final diagrams
        print(f"\nFinal diagrams generated: {len(final_state['diagrams'])}")
        for key, diagram in final_state['diagrams'].items():
            print(f"\n🎯 Diagram: {key}")
            print(f"Length: {len(diagram)} characters")
            print('### Full Diagram ###')
            for i, line in enumerate(diagram.splitlines(), start=1):
                print(f"{i:03d}: {line}")
            print('######################')
            
            # Validate the final diagram with new approach
            final_parse_errors = _parser_errors_python_only(diagram)
            final_warnings = (
                _edge_warnings(diagram) + 
                _label_warnings(diagram) + 
                _node_id_warnings(diagram) + 
                _complexity_warnings(diagram)
            )
            
            print(f"Final parse errors: {len(final_parse_errors)}")
            print(f"Final warnings: {len(final_warnings)}")
            
            if final_parse_errors:
                print("❌ Final diagram has blocking errors:")
                for error in final_parse_errors:
                    print(f"  - {error}")
            else:
                print("✅ Final diagram has no blocking errors!")
            
            if final_warnings:
                print("⚠️ Final diagram warnings:")
                for warning in final_warnings[:3]:  # Show first 3 warnings
                    print(f"  - {warning}")
            else:
                print("✅ Final diagram has no warnings!")
        
        return final_state
        
    except Exception as e:
        print(f"❌ Error during subgraph execution: {e}")
        import traceback
        traceback.print_exc()
        return None

# Test different modes
modes_to_test = ["balanced", "simple", "detailed"]

results = {}
for mode in modes_to_test:
    print(f"\n{'='*80}")
    print(f"TESTING MODE: {mode.upper()}")
    print(f"{'='*80}")
    
    result = test_diagram_subgraph(mode)
    results[mode] = result
    
    print(f"\n✅ Completed testing mode: {mode}")



TESTING MODE: BALANCED
🎨 Testing Full Diagram Subgraph - Mode: balanced

Initial state created for mode: balanced
Max attempts: 3
2025-09-21 20:32:13,871 - workflows.diagram_subgraph - INFO - 🎨 Generating balanced diagram (attempt 1)
2025-09-21 20:32:13,873 - backend.llm.retry_handler - INFO - 🤖 Trying model: gemini-2.5-flash-lite
2025-09-21 20:32:18,135 - workflows.diagram_subgraph - INFO - ✅ LLM diagram generated
2025-09-21 20:32:18,141 - workflows.diagram_subgraph - INFO - Validating mermaid diagram
2025-09-21 20:32:18,146 - workflows.diagram_subgraph - INFO - Validation passed
2025-09-21 20:32:18,149 - workflows.diagram_subgraph - INFO - 3 lint warning(s) that do not block rendering
2025-09-21 20:32:18,161 - workflows.diagram_subgraph - INFO - 🏁 Finalizing balanced diagram
2025-09-21 20:32:18,164 - workflows.diagram_subgraph - INFO - ✅ Using validated diagram

📊 Subgraph execution completed!
Final attempt: 1
Is valid: True
Validation errors: 0
Validation warnings: 3

⚠️ Non-blocki

In [5]:
## Test Specific Edge Cases

# Let's test specific cases that should now pass validation with our new permissive approach:


In [6]:
# Test specific edge cases that should now pass validation
edge_case_tests = {
    "quoted_edge_labels": """
graph LR
    A[main.py] -- "Uses configuration" --> B[config.py]
    A --|"Helper functions"|--> C[utils.py]
    B -.-> D[helpers.py]
""".strip(),
    
    "special_characters": """
graph LR
    A[main.py & utils] --> B[config < settings]
    A --> C["quoted & special"]
    B --> D[helpers.py]
""".strip(),
    
    "various_arrow_styles": """
graph LR
    A[main.py] --> B[utils.py]
    A -.-> C[config.py]
    A ==> D[database.py]
    A --o E[cache.py]
    A o-- F[session.py]
""".strip(),
    
    "flowchart_with_init": """
%%{init: {'theme':'base', 'themeVariables': { 'primaryColor': '#ff0000'}}}%%
flowchart LR
    A[main.py] --> B[utils.py]
    A --> C[config.py]
""".strip(),
    
    "subgraph_with_labels": """
graph LR
    subgraph "Main Module"
        A[main.py] --> B[utils.py]
    end
    subgraph "Config Module"
        C[config.py]
    end
    A --> C
""".strip()
}

print("🔍 Testing Edge Cases That Should Now Pass\n")

for name, diagram in edge_case_tests.items():
    print(f"\n📊 Testing: {name}")
    print(f"Diagram preview: {diagram[:80]}...")
    
    # Test with new validation approach
    parse_errors = _parser_errors_python_only(diagram)
    warnings = (
        _edge_warnings(diagram) + 
        _label_warnings(diagram) + 
        _node_id_warnings(diagram) + 
        _complexity_warnings(diagram)
    )
    
    print(f"  Parse errors (blocking): {len(parse_errors)}")
    print(f"  Warnings (non-blocking): {len(warnings)}")
    
    if parse_errors:
        print("  ❌ Still has blocking errors:")
        for error in parse_errors:
            print(f"    - {error}")
    else:
        print("  ✅ No blocking errors - should render!")
    
    if warnings:
        print("  ⚠️ Has warnings:")
        for warning in warnings[:2]:  # Show first 2 warnings
            print(f"    - {warning}")
    else:
        print("  ✅ No warnings!")
    
    print("-" * 50)

print("\n🎯 Summary: These edge cases should now pass validation with our new permissive approach!")


🔍 Testing Edge Cases That Should Now Pass


📊 Testing: quoted_edge_labels
Diagram preview: graph LR
    A[main.py] -- "Uses configuration" --> B[config.py]
    A --|"Helpe...
  Parse errors (blocking): 0
  Warnings (non-blocking): 3
  ✅ No blocking errors - should render!
  ⚠️ Has warnings:
    - Line 2: edge looks unusual for flowchart syntax
    - Line 3: edge looks unusual for flowchart syntax
--------------------------------------------------

📊 Testing: special_characters
Diagram preview: graph LR
    A[main.py & utils] --> B[config < settings]
    A --> C["quoted & s...
  Parse errors (blocking): 0
  Warnings (non-blocking): 6
  ✅ No blocking errors - should render!
  ⚠️ Has warnings:
    - Line 2: edge looks unusual for flowchart syntax
    - Line 3: edge looks unusual for flowchart syntax
--------------------------------------------------

📊 Testing: various_arrow_styles
Diagram preview: graph LR
    A[main.py] --> B[utils.py]
    A -.-> C[config.py]
    A ==> D[data...
  Parse

In [7]:
## Test Auto-Correction Functionality

# Let's test the updated auto-correction function with safe operations only:


In [8]:
# Test auto-correction functionality
correction_tests = {
    "missing_header": """
    A[main.py] --> B[utils.py]
    A --> C[config.py]
""".strip(),
    
    "unbalanced_subgraph": """
graph LR
    subgraph "Main Module"
        A[main.py] --> B[utils.py]
    end
    subgraph "Config Module"
        C[config.py]
    end
""".strip(),
    
    "very_long_label": """
graph LR
    A[This is an extremely long label that should be truncated because it exceeds the reasonable length limit for diagram labels and might cause rendering issues] --> B[utils.py]
""".strip()
}

print("🔧 Testing Auto-Correction Functionality\n")

for name, diagram in correction_tests.items():
    print(f"\n📊 Testing: {name}")
    print(f"Original diagram:")
    print(diagram)
    print()
    
    # Get errors first
    parse_errors = _parser_errors_python_only(diagram)
    print(f"Parse errors found: {len(parse_errors)}")
    for error in parse_errors:
        print(f"  - {error}")
    
    # Apply corrections
    corrected_diagram = diagram
    for error in parse_errors:
        corrected_diagram = _apply_correction(corrected_diagram, error)
    
    if corrected_diagram != diagram:
        print(f"\n✅ Diagram was corrected!")
        print("Corrected diagram:")
        print(corrected_diagram)
        
        # Check if corrections worked
        new_errors = _parser_errors_python_only(corrected_diagram)
        print(f"\nParse errors after correction: {len(new_errors)}")
        if new_errors:
            print("Remaining errors:")
            for error in new_errors:
                print(f"  - {error}")
        else:
            print("✅ All blocking errors fixed!")
    else:
        print("⚠️ No corrections were applied")
    
    print("-" * 50)

print("\n🎯 Summary: Auto-correction should only fix safe operations like missing headers and subgraph balance!")


🔧 Testing Auto-Correction Functionality


📊 Testing: missing_header
Original diagram:
A[main.py] --> B[utils.py]
    A --> C[config.py]

Parse errors found: 1
  - Missing diagram type declaration. Add 'flowchart LR' or 'graph TD' at the top.

✅ Diagram was corrected!
Corrected diagram:
flowchart LR
A[main.py] --> B[utils.py]
    A --> C[config.py]

Parse errors after correction: 0
✅ All blocking errors fixed!
--------------------------------------------------

📊 Testing: unbalanced_subgraph
Original diagram:
graph LR
    subgraph "Main Module"
        A[main.py] --> B[utils.py]
    end
    subgraph "Config Module"
        C[config.py]
    end

Parse errors found: 0
⚠️ No corrections were applied
--------------------------------------------------

📊 Testing: very_long_label
Original diagram:
graph LR
    A[This is an extremely long label that should be truncated because it exceeds the reasonable length limit for diagram labels and might cause rendering issues] --> B[utils.py]

Parse err